### 5회 기출 - 작업2유형
- 벤츠 차량 가격 예측 : https://www.kaggle.com/datasets/mysarahmadbhat/mercedes-used-car-listing 
- train = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv 
- test = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv
- 예측 변수 price, test.csv에 대해 price 값을 예측하여 제출, 제출 데이터 컬럼은 price만 존재해야함.
- 평가지표는 rmse

In [ ]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv')

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

풀이!

In [ ]:
import sklearn.metrics
print(dir(sklearn.metrics)[65:])

['matthews_corrcoef', 'max_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_gamma_deviance', 'mean_pinball_loss', 'mean_poisson_deviance', 'mean_squared_error', 'mean_squared_log_error', 'mean_tweedie_deviance', 'median_absolute_error', 'multilabel_confusion_matrix', 'mutual_info_score', 'nan_euclidean_distances', 'ndcg_score', 'normalized_mutual_info_score', 'pair_confusion_matrix', 'pairwise', 'pairwise_distances', 'pairwise_distances_argmin', 'pairwise_distances_argmin_min', 'pairwise_distances_chunked', 'pairwise_kernels', 'precision_recall_curve', 'precision_recall_fscore_support', 'precision_score', 'r2_score', 'rand_score', 'recall_score', 'roc_auc_score', 'roc_curve', 'silhouette_samples', 'silhouette_score', 'top_k_accuracy_score', 'v_measure_score', 'zero_one_loss']


In [ ]:
# mape는 실제값(y_true)이 0인 경우 또는 0에 가까운 작은 값인 경우는 사용할 수 없음
from sklearn.metrics import mean_absolute_percentage_error as mape

#mape([1, 10, 100], [1, 9, 90])  # 0%  10% error  10% error
mape([1, 0, 100], [1, 9, 90])

1.3510798882111488e+16

In [ ]:
import numpy as np

A = np.array([0.5**2, 1 ** 2, 2 ** 2, 4 ** 2, 5 ** 2]) ** 0.5
B = np.sqrt(np.array([0.5**2, 1 ** 2, 2 ** 2, 4 ** 2, 5 ** 2]))
print(A, B)

[0.5 1.  2.  4.  5. ] [0.5 1.  2.  4.  5. ]


In [ ]:
import numpy as np
np.log(-1)

<ipython-input-34-07ff17fea371>:2: RuntimeWarning: invalid value encountered in log
  np.log(-1)


nan

In [ ]:
#[0] import
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression   # LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as mae
# mape는 실제값(y_true)이 0인 경우는 사용할 수 없음
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
# msle, rmsle에는 음수를 사용할 수 없음 
from sklearn.metrics import mean_squared_log_error as msle
# mse ** 0.5 = np.sqrt(mse) => rmse
# msle ** 0.5 = np.sqrt(msle) => rmsle


def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)  # r2_score (결정계수)
    y_pred = model.predict(x_test)
    y_pred = np.where(y_pred >= 0, y_pred, -y_pred)  # 음수보정 (절대값, 0, ...)
    C = mae(y_test, y_pred)
    D = mse(y_test, y_pred)
    E = mse(y_test, y_pred) ** 0.5   # D ** 0.5
    F = msle(y_test, y_pred)
    G = msle(y_test, y_pred) ** 0.5  # F ** 0.5
    return f'R2: {A:.4f} {B:.4f} mae: {C:.4f} mse: {D:.4f} rmse: {E:.4f} msle: {F:.4f} rmsle: {G:.4f}'

# [1] 파일 불러오기

XY = pd.read_csv('train.csv')
X_submission = pd.read_csv('test.csv')
print(XY.shape, X_submission.shape)

#print(XY.head(2))
#print(X_submission.head(2))

#print(XY.nunique())
#print(X_submission.nunique())
#print(XY['model'].value_counts())
#print(X_submission['model'].value_counts())
#set(XY['model']) - set(X_submission['model'])  # {' CLC Class', ' R Class', '180', '200', '230'}
#set(X_submission['model']) - set(XY['model'])  # {'220'}

# [2] X, Y_use 분리,  X, X_submission 연결 -> dfX 생성
#print(XY.head(2))
X = XY.drop(columns=['price'])
Y_use = XY['price']
dfX = pd.concat([X, X_submission], axis=0)

# [3] 데이터 탐색
#dfX.info()  # object : 'model', 'transmission', 'fuelType'
#print(X.describe())            # numeric dtype
#print(X_submission.describe())

# 범주형 변수에 대해서는 nunique가 같은지, unique가 같은지 확인
# 실제 범주 종류도 같은지 알아보도록 한다. 만일 X_submission에 있는 범주가 X에 없다면 오류!
#C_features = ['model', 'transmission', 'fuelType']
#print(X[C_features].nunique())
#print(X_submission[C_features].nunique())
# LABEL Encoding
feature = ['model', 'transmission', 'fuelType']
for x in feature:
    dfX[x] = dfX[x].astype('category').cat.codes
#dfX.info()
#print(dfX.columns)
# OneHot Encoding
#dfX = pd.get_dummies(dfX)
#dfX.info()

# X, X_submission 을 각각 OneHot Encoding, 범주 불일치에 대해 reindex함수 사용
#dum_X = pd.get_dummies(X)
#dum_X_submission = pd.get_dummies(X_submission)
#dum_X_submission = dum_X_submission.reindex(columns = dum_X.columns, fill_value=0)
#dum_X.info()

# [4] 스케일러
dfX = pd.DataFrame(StandardScaler().fit_transform(dfX))
# dfX ->  AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

# [5] X_use, X_submission (분리) 
X_use = dfX.iloc[:9823, :]
X_submission = dfX.iloc[9823:, :]
#print(X_use.shape, X_submission.shape)

# [6] train, test 분리
temp = train_test_split(X_use, Y_use, test_size=0.2, random_state=1) #, stratify=Y_use)
# ValueError: The least populated class in y has only 1 member, which is too few. 
# The minimum number of groups for any class cannot be less than 2.
x_train, x_test, y_train, y_test = temp
#print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# [7] 모델링 (모델 선택, 학습, 평가)
model1 = LinearRegression()
model1.fit(x_train, y_train)
print(get_scores(model1, x_train, x_test, y_train, y_test))

##  모델링

# 데이터마님 문제 풀이, pandas_03, pandas_04 학습
# Python, VSCode, 데이터마님 사용법 - 영상
# 통계 분석 - 영상 

(9823, 10) (3296, 9)
R2: 0.7118 0.6632 mae: 3819.5734 mse: 37650908.7148 rmse: 6136.0336 msle: 0.0965 rmsle: 0.3107


In [ ]:
!pip install shelves

In [ ]:
# 실무에서 Encoder 객체를 저장해 두었다 사용하는 방법
import pandas as pd
from sklearn.preprocessing import LabelEncoder

feature = ['model', 'transmission', 'fuelType']
encoders = pd.Series()
for x in feature:
    encoder = LabelEncoder()
    dfX[x] = encoder.fit_transform(dfX[x])
    encoders[x] = encoder


import shelve
with shelve.open('mercedes') as data:
    data['encoders'] = encoders

# X, X_submission 을 각각 OneHot Encoding, 범주 불일치에 대해 reindex함수 사용
# 주기적으로 모델을 갱신 (최신 데이터를 반영해서 새로운 모델 만들기)    

In [ ]:
set(train['model'].str.strip())

{'180',
 '200',
 '230',
 'A Class',
 'B Class',
 'C Class',
 'CL Class',
 'CLA Class',
 'CLC Class',
 'CLK',
 'CLS Class',
 'E Class',
 'G Class',
 'GL Class',
 'GLA Class',
 'GLB Class',
 'GLC Class',
 'GLE Class',
 'GLS Class',
 'M Class',
 'R Class',
 'S Class',
 'SL CLASS',
 'SLK',
 'V Class',
 'X-CLASS'}

In [ ]:
set(X_submission['model'])

{' A Class',
 ' B Class',
 ' C Class',
 ' CL Class',
 ' CLA Class',
 ' CLK',
 ' CLS Class',
 ' E Class',
 ' G Class',
 ' GL Class',
 ' GLA Class',
 ' GLB Class',
 ' GLC Class',
 ' GLE Class',
 ' GLS Class',
 ' M Class',
 ' S Class',
 ' SL CLASS',
 ' SLK',
 ' V Class',
 ' X-CLASS',
 '220'}

In [ ]:
# 범주불일치
# X에는 없고, X_submission에만 있는 경우 LabelEncoding, OneHotEncoding도 문제를 야기할 수 있음
df1 = pd.DataFrame({'blood_type': ['A', 'B', 'O', 'A', 'O', 'B']})
df1['blood_type'].astype('category').cat.codes  # 'A': 0, 'B':1, 'O': 2

In [ ]:
df2 = pd.DataFrame({'blood_type': ['A', 'B', 'AB', 'A', 'O', 'B']})
df2['blood_type'].astype('category').cat.codes  # 'A': 0, 'AB':1, 'B':2, 'O': 3

In [ ]:
dfX = pd.concat([df1, df2])
dfX = pd.get_dummies(dfX)
dfX.shape

(12, 4)